In [1]:
import datetime
import tarfile

import boto3
import pandas as pd
import numpy as np
from sagemaker import get_execution_role
import sagemaker



sm_boto3 = boto3.client('sagemaker')

sess = sagemaker.Session()

region = sess.boto_session.region_name

bucket = sess.default_bucket()  # this could also be a hard-coded bucket name

print('Using bucket ' + bucket)

Using bucket sagemaker-us-east-1-754307369999


In [2]:
import pandas as pd
from sagemaker.sklearn.estimator import SKLearn
import time
import boto3
import json
import numpy as np
import pandas as pd
import sagemaker
from sagemaker.session import Session
from io import StringIO
import time

sess = boto3.Session()
sm = sess.client('sagemaker')
role = sagemaker.get_execution_role()
data=pd.read_excel("s3://sagemaker-us-east-1-754307369999/TrainData/DatasetEntry.xlsx",encoding = "ISO-8859-1",error_bad_lines=False)
filename=data['filename'].iloc[0]
Type=data['Type'].iloc[0]
model=data['Model'].iloc[0]
print(model)

DatasetPath=data['DatasetPath'].iloc[0]

Autopilot


In [3]:
import pandas as pd
from sagemaker.sklearn.estimator import SKLearn
import time
import sagemaker
from sagemaker.session import Session
!pip install xlrd
!pip install s3fs


sess = boto3.Session()
sm = sess.client('sagemaker')
role = sagemaker.get_execution_role()
data=pd.read_excel("s3://sagemaker-us-east-1-754307369999/TrainData/DatasetEntry.xlsx",encoding = "ISO-8859-1",error_bad_lines=False)
print(data['DatasetPath'].iloc[0])
print(data['Model'].iloc[0])
print(data['Type'].iloc[0])
print(data['filename'].iloc[0])
filename=data['filename'].iloc[0]
Type=data['Type'].iloc[0]
model=data['Model'].iloc[0]
local_data_path=data['DatasetPath'].iloc[0]
T_attribute=data['Attribute'].iloc[0]
print(T_attribute)
DatasetName=data['DatasetName'].iloc[0]
print(DatasetName)
data = pd.read_csv(local_data_path)
pd.set_option('display.max_columns', 500)     # Make sure we can see all of the columns
pd.set_option('display.max_rows', 10)         # Keep the output on one page
data

You are using pip version 10.0.1, however version 20.0.2 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
You are using pip version 10.0.1, however version 20.0.2 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
s3://sagemaker-us-east-1-754307369999/TrainData/loandataset.csv
Autopilot
Train
loandataset.csv
Loan_Status
LoanPrediction


,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,Male,No,0,Graduate,No,5849,0.0,NaN,360.0,1.0,Urban,Y
1,Male,Yes,1,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural,N
2,Male,Yes,0,Graduate,Yes,3000,0.0,66.0,360.0,1.0,Urban,Y
3,Male,Yes,0,Not Graduate,No,2583,2358.0,120.0,360.0,1.0,Urban,Y
4,Male,No,0,Graduate,No,6000,0.0,141.0,360.0,1.0,Urban,Y
...,...,...,...,...,...,...,...,...,...,...,...,...
1616,Female,No,0,Graduate,No,2900,0.0,71.0,360.0,1.0,Rural,Y
1617,Male,Yes,3+,Graduate,No,4106,0.0,40.0,180.0,1.0,Rural,Y
1618,Male,Yes,1,Graduate,No,8072,240.0,253.0,360.0,1.0,Urban,Y
1619,Male,Yes,2,Graduate,No,7583,0.0,187.0,360.0,1.0,Urban,Y


In [27]:
train_data = data.sample(frac=0.8,random_state=200)

test_data = data.drop(train_data.index)

test_data_no_target = test_data.drop(columns=[T_attribute])

In [28]:

import sagemaker
import boto3
from sagemaker import get_execution_role

region = boto3.Session().region_name
prefix=DatasetName
session = sagemaker.Session()
train_file = 'train_data.csv';
train_data.to_csv(train_file, index=False, header=True)
train_data_s3_path = session.upload_data(path=train_file, key_prefix=prefix + "/train")
print('Train data uploaded to: ' + train_data_s3_path)

test_file = 'test_data.csv';
test_data_no_target.to_csv(test_file, index=False, header=False)
test_data_s3_path = session.upload_data(path=test_file, key_prefix=prefix + "/test")
print('Test data uploaded to: ' + test_data_s3_path)

Train data uploaded to: s3://sagemaker-us-east-1-754307369999/LoanPrediction/train/train_data.csv
Test data uploaded to: s3://sagemaker-us-east-1-754307369999/LoanPrediction/test/test_data.csv


In [29]:
input_data_config = [{
      'DataSource': {
        'S3DataSource': {
          'S3DataType': 'S3Prefix',
          'S3Uri': 's3://{}/{}/train'.format(bucket,prefix)
        }
      },
      'TargetAttributeName': T_attribute
    }
  ]

output_data_config = {
    'S3OutputPath': 's3://{}/{}/output'.format(bucket,prefix)
  }

In [33]:
from time import gmtime, strftime, sleep
timestamp_suffix = strftime('%d-%H-%M-%S', gmtime())

auto_ml_job_name = 'automl-Generic-'+DatasetName
print('AutoMLJobName: ' + auto_ml_job_name)

sm.create_auto_ml_job(AutoMLJobName=auto_ml_job_name,
                      InputDataConfig=input_data_config,
                      OutputDataConfig=output_data_config,
                      RoleArn=role)

AutoMLJobName: automl-Generic-LoanPrediction


{'AutoMLJobArn': 'arn:aws:sagemaker:us-east-1:754307369999:automl-job/automl-generic-loanprediction',
 'ResponseMetadata': {'RequestId': '4b2410a4-6736-4bf0-8a5f-6e3346d2307a',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '4b2410a4-6736-4bf0-8a5f-6e3346d2307a',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '100',
   'date': 'Mon, 06 Apr 2020 17:07:36 GMT'},
  'RetryAttempts': 0}}

In [ ]:
print ('JobStatus - Secondary Status')
print('------------------------------')


describe_response = sm.describe_auto_ml_job(AutoMLJobName=auto_ml_job_name)
print (describe_response['AutoMLJobStatus'] + " - " + describe_response['AutoMLJobSecondaryStatus'])
job_run_status = describe_response['AutoMLJobStatus']
    
while job_run_status not in ('Failed', 'Completed', 'Stopped'):
    describe_response = sm.describe_auto_ml_job(AutoMLJobName=auto_ml_job_name)
    job_run_status = describe_response['AutoMLJobStatus']
    
    print (describe_response['AutoMLJobStatus'] + " - " + describe_response['AutoMLJobSecondaryStatus'])
    sleep(30)

JobStatus - Secondary Status
------------------------------
InProgress - AnalyzingData
InProgress - AnalyzingData
InProgress - AnalyzingData
InProgress - AnalyzingData
InProgress - AnalyzingData
InProgress - AnalyzingData
InProgress - AnalyzingData
InProgress - AnalyzingData
InProgress - AnalyzingData
InProgress - AnalyzingData
InProgress - AnalyzingData
InProgress - AnalyzingData
InProgress - AnalyzingData
InProgress - AnalyzingData
InProgress - AnalyzingData
InProgress - AnalyzingData
InProgress - AnalyzingData
InProgress - AnalyzingData
InProgress - FeatureEngineering
InProgress - FeatureEngineering
InProgress - FeatureEngineering
InProgress - FeatureEngineering
InProgress - FeatureEngineering
InProgress - FeatureEngineering
InProgress - FeatureEngineering
InProgress - FeatureEngineering
InProgress - FeatureEngineering
InProgress - FeatureEngineering
InProgress - FeatureEngineering
InProgress - FeatureEngineering
InProgress - FeatureEngineering
InProgress - FeatureEngineering
InProg

In [6]:
sm = boto3.Session().client(service_name='sagemaker',region_name='us-east-1')
AutoMLJobName = 'automl-Generic-'+DatasetName
model_name=AutoMLJobName
best_candidate = sm.describe_auto_ml_job(AutoMLJobName=AutoMLJobName)['BestCandidate']
best_candidate_name = best_candidate['CandidateName']
print(best_candidate)
print('\n') 
print("CandidateName: " + best_candidate_name)
try:
    model = sm.create_model(Containers=best_candidate['InferenceContainers'],
                            ModelName=model_name,
                            ExecutionRoleArn='arn:aws:iam::754307369999:role/service-role/AmazonSageMaker-ExecutionRole-20200118T204448')
        
    print('Model ARN corresponding to the best candidate is : {}'.format(model['ModelArn']))
    client = boto3.client('sagemaker')
    response = client.create_endpoint_config(
    EndpointConfigName=DatasetName,
    ProductionVariants=[
        {
            'VariantName': 'default-variant-name',
            'ModelName': model_name ,
            'InitialInstanceCount': 1,
            'InstanceType': 'ml.t2.medium'        },
        ],
        DataCaptureConfig={
            'EnableCapture': False,
            'InitialSamplingPercentage': 50,
            'DestinationS3Uri': 's3://sagemaker-us-east-2-754307369999/Testdata',
            'CaptureOptions': [
                {
                    'CaptureMode': 'Input'
                },
        ],
                
    }
    )
            
    print(response)
except:
    client = boto3.client('sagemaker')
    response = client.create_endpoint(
    EndpointName="BankingSA-Endpoint"+DatasetName,
    EndpointConfigName=DatasetName)
    print(response)
     

[{'Image': '683313688378.dkr.ecr.us-east-1.amazonaws.com/sagemaker-sklearn-automl:0.1.0-cpu-py3', 'ModelDataUrl': 's3://sagemaker-us-east-1-754307369999/LoanPrediction/output/automl-Generic-LoanPrediction/data-processor-models/automl-Gen-dpp1-1-f86d4e7b70624f82bac699c1dd9b30ec3a90474f79184/output/model.tar.gz', 'Environment': {'AUTOML_TRANSFORM_MODE': 'feature-transform', 'SAGEMAKER_DEFAULT_INVOCATIONS_ACCEPT': 'application/x-recordio-protobuf', 'SAGEMAKER_PROGRAM': 'sagemaker_serve', 'SAGEMAKER_SUBMIT_DIRECTORY': '/opt/ml/model/sagemaker_serve.py'}}, {'Image': '683313688378.dkr.ecr.us-east-1.amazonaws.com/sagemaker-xgboost:0.90-1-cpu-py3', 'ModelDataUrl': 's3://sagemaker-us-east-1-754307369999/LoanPrediction/output/automl-Generic-LoanPrediction/tuning/automl-Gen-dpp1-xgb/tuning-job-1-eaad5fe5f17143609b-212-4e6f4d03/output/model.tar.gz', 'Environment': {'MAX_CONTENT_LENGTH': '20971520', 'SAGEMAKER_DEFAULT_INVOCATIONS_ACCEPT': 'text/csv'}}, {'Image': '683313688378.dkr.ecr.us-east-1.amaz

NameError: name 'DatasetName' is not defined